<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
# import piplite
# await piplite.install(['folium'])
# await piplite.install(['pandas'])

In [ ]:
# !pip install js

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
# from js import fetch
# import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
# resp = await fetch(URL)
# spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
# spacex_df=pd.read_csv(spacex_csv_file)
spacex_df = pd.read_csv(URL)

In [6]:
spacex_df.columns

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long'],
      dtype='object')

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
spacex_df.tail(10)

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    coordinate = [record['Lat'], record['Long']]
    if record['class'] == 1:
        marker_color = 'green'
    else:
        marker_color = 'red'
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=marker_color)
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [8]:
# TASK 1
# Create a base map centered on the average coordinates of all sites
site_coords = spacex_df.groupby('Launch Site')[['Lat', 'Long']].mean()
map_center = [site_coords['Lat'].mean(), site_coords['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each unique launch site with site names
for site, coords in site_coords.iterrows():
    folium.Marker(
        location=[coords['Lat'], coords['Long']],
        popup=folium.Popup(
            f"<b>{site}</b><br>Coordinates: {coords['Lat']:.4f}, {coords['Long']:.4f}",
            max_width=300
        ),
        tooltip=f"{site}",  # Shows site name on hover
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

# Display the map
site_map

In [9]:
# TASK 2
# Create a marker cluster for launches
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch, colored by success/failure
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    marker_color = 'green' if row['class'] == 1 else 'red'
    
    folium.Marker(
        location=coordinate,
        popup=f"Launch: {row.get('Launch Site', 'Unknown')} - {'Success' if row['class'] == 1 else 'Failure'}",
        tooltip=f"{'Success' if row['class'] == 1 else 'Failure'} Launch",
        icon=folium.Icon(color='white', icon_color=marker_color, icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Add simple legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 90px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
<h4>Legend</h4>
<p><i class="fa fa-rocket" style="color:green"></i> Success</p>
<p><i class="fa fa-rocket" style="color:red"></i> Failure</p>
</div>
'''
site_map.get_root().html.add_child(folium.Element(legend_html))

# Display the updated map
site_map

In [11]:
#TASK 3
from folium import plugins
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Haversine formula to calculate distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    distance_km = R * c
    distance_miles = distance_km * 0.621371
    
    return round(distance_km, 2), round(distance_miles, 2)

# Define proximity points (coastline points, major cities, other sites)
proximity_points = {
    'Major Cities': {
        'Los Angeles, CA': (34.0522, -118.2437),
        'San Francisco, CA': (37.7749, -122.4194),
        'Miami, FL': (25.7617, -80.1918),
        'New York, NY': (40.7128, -74.0060),
        'Houston, TX': (29.7604, -95.3698)
    },
    'Other Sites': {
        'Vandenberg SLC': (34.6325, -120.6108),
        'Cape Canaveral': (28.3922, -80.6077),
        'Kennedy Space Center': (28.5214, -80.6503),
        'Brownsville (Starbase)': (25.9971, -97.1558)
    }
}

# Get unique launch sites with coordinates
site_coords = spacex_df.groupby('Launch Site')[['Lat', 'Long']].mean().reset_index()

# Create enhanced map
site_map_proximity = folium.Map(location=[28.5, -80.5], zoom_start=6, tiles='OpenStreetMap')

# Create marker cluster for launch markers
marker_cluster = MarkerCluster().add_to(site_map_proximity)

# Add individual launch markers (green/red)
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    marker_color = 'green' if row['class'] == 1 else 'red'
    
    folium.Marker(
        location=coordinate,
        popup=f"Launch: {row.get('Launch Site', 'Unknown')} - {'Success' if row['class'] == 1 else 'Failure'}",
        tooltip=f"{'Success' if row['class'] == 1 else 'Failure'} Launch",
        icon=folium.Icon(color='white', icon_color=marker_color, icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Add launch site summary markers and proximity distance lines
for idx, site_row in site_coords.iterrows():
    site_name = site_row['Launch Site']
    site_lat, site_lon = site_row['Lat'], site_row['Long']
    
    # Create main site marker
    folium.Marker(
        location=[site_lat, site_lon],
        popup=f"<b>{site_name}</b><br>Click for proximity analysis",
        tooltip=site_name,
        icon=folium.Icon(color='blue', icon='map-marker-alt', prefix='fa', icon_size=(25,41))
    ).add_to(site_map_proximity)
    
    # Distance analysis popup
    distances_html = "<h4>📏 Distance Analysis</h4><table style='width:100%; border-collapse: collapse;'>"
    
    # Calculate distances to proximity points
    for category, points in proximity_points.items():
        distances_html += f"<tr><td colspan='3'><b>{category}</b></td></tr>"
        
        for point_name, (point_lat, point_lon) in points.items():
            km, miles = haversine_distance(site_lat, site_lon, point_lat, point_lon)
            
            # Color code based on distance (closer = greener)
            if km < 100:
                color = 'green'
            elif km < 500:
                color = 'orange'
            else:
                color = 'red'
            
            distances_html += f"""
            <tr style="background-color: #f9f9f9;">
                <td style="padding: 5px;">{point_name}</td>
                <td style="padding: 5px; color: {color}; font-weight: bold;">{km} km</td>
                <td style="padding: 5px; font-size: 11px;">({miles} mi)</td>
            </tr>
            """
    
    distances_html += "</table>"
    
    # Add site marker with distance popup
    folium.Marker(
        location=[site_lat, site_lon],
        popup=folium.Popup(distances_html, max_width=350),
        tooltip=f"{site_name} - Distances",
        icon=folium.DivIcon(
            html=f'<div style="background-color: blue; color: white; padding: 5px; border-radius: 50%; font-weight: bold; font-size: 12px;">{site_name[:3].upper()}</div>'
        )
    ).add_to(site_map_proximity)
    
    # Draw distance lines to nearby points (within 1000km)
    for category, points in proximity_points.items():
        for point_name, (point_lat, point_lon) in points.items():
            km, _ = haversine_distance(site_lat, site_lon, point_lat, point_lon)
            
            if km <= 1000:  # Only show lines for nearby points
                # Create line
                folium.PolyLine(
                    locations=[[site_lat, site_lon], [point_lat, point_lon]],
                    weight=2,
                    color='purple',
                    opacity=0.6,
                    popup=f"{site_name} → {point_name}<br>{km} km"
                ).add_to(site_map_proximity)
                
                # Add proximity point marker
                folium.CircleMarker(
                    location=[point_lat, point_lon],
                    radius=4,
                    popup=f"{point_name}<br>{km} km from {site_name}",
                    color='purple',
                    fill=True,
                    fillColor='purple',
                    fillOpacity=0.7,
                    tooltip=point_name
                ).add_to(site_map_proximity)

# Add distance measurement tool
site_map_proximity.add_child(plugins.MeasureControl(
    primary_length_unit='kilometers',
    secondary_length_unit='miles',
    primary_area_unit='sqkilometers',
    secondary_area_unit='sqmiles'
))

# Enhanced legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 250px; height: 180px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:12px; padding: 15px; border-radius: 5px;">
<h4><b>🚀 Launch Site Map</b></h4>
<p><i class="fa fa-rocket" style="color:green"></i> <b>Success Launch</b></p>
<p><i class="fa fa-rocket" style="color:red"></i> <b>Failed Launch</b></p>
<p><i class="fa fa-map-marker-alt" style="color:blue"></i> <b>Launch Site</b></p>
<hr style="margin: 8px 0;">
<p><i class="fa fa-circle" style="color:purple"></i> <b>Proximity Point</b></p>
<p><span style="border: 2px solid purple; display: inline-block; width: 20px; height: 2px;"></span> <b>Distance Line</b></p>
<p><small>Click sites for distance tables!</small></p>
</div>
'''
site_map_proximity.get_root().html.add_child(folium.Element(legend_html))

# Display the proximity analysis map
site_map_proximity

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Get unique launch sites and their coordinates (mean if multiple per site)
unique_sites = spacex_df.groupby('Launch Site')[['Lat', 'Long']].mean().reset_index()

# Calculate pairwise distances
distances = pd.DataFrame(index=unique_sites['Launch Site'], columns=unique_sites['Launch Site'])
for i, site1 in enumerate(unique_sites['Launch Site']):
    for j, site2 in enumerate(unique_sites['Launch Site']):
        if i != j:  # Exclude distance to itself
            dist = calculate_distance(
                unique_sites.iloc[i]['Lat'],
                unique_sites.iloc[i]['Long'],
                unique_sites.iloc[j]['Lat'],
                unique_sites.iloc[j]['Long']
            )
            distances.loc[site1, site2] = round(dist, 2)
        else:
            distances.loc[site1, site2] = 0

# Display the distance matrix
print("Distance Matrix (km) between Launch Sites:")
print(distances)

# Optional: For a specific site, get distances to all proximities
specific_site = 'CCAFS LC-40'  # Replace with your target site
proximity_distances = distances[specific_site].dropna().sort_values()
print(f"\nDistances from {specific_site} to other sites:")
print(proximity_distances)

In [ ]:
# Add MousePosition plugin to the map for selecting coastline coordinates
site_map.add_child(MousePosition(
    prefix='Mouse Position:',
    separator=' | '
))

# Define launch site coordinates (e.g., for CCAFS LC-40)
launch_coordinate = [28.56194, -80.57722]

# Selected closest coastline point (e.g., via MousePosition hover on map; approximate Atlantic coast point)
coordinate = [28.56194, -80.55000]

# Calculate distance using Haversine formula
distance = calculate_distance(launch_coordinate[0], launch_coordinate[1], coordinate[0], coordinate[1])

# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)
distance_marker.add_to(site_map)

# Draw a PolyLine between the launch site and the selected coastline point
folium.PolyLine(
    locations=[launch_coordinate, coordinate],
    weight=2,
    color='blue',
    popup='Distance to Coastline: {:.2f} KM'.format(distance)
).add_to(site_map)

# Display the updated map
site_map

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
#distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
       )
   )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# Define the launch site and coastline coordinates (example for CCAFS LC-40)
launch_coordinate = [28.56194, -80.57722]  # Launch site coordinates
coastline_coordinate = [28.56194, -80.55000]  # Closest coastline point coordinates

# Create a list of coordinates for the PolyLine
coordinates = [launch_coordinate, coastline_coordinate]

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,  # Increased weight for better visibility
    color='blue',  # Added color for distinction
    popup='Distance to Coastline: {:.2f} KM'.format(calculate_distance(launch_coordinate[0], launch_coordinate[1], coastline_coordinate[0], coastline_coordinate[1]))  # Added popup with distance
)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
